# DataJoint U24 - Workflow DeepLabCut

## Workflow Automation

In the previous notebook [03-Process](./03-Process.ipynb), we ran through the workflow in detailed steps. For daily running routines, the current notebook provides a more succinct and automatic approach to run through the pipeline using some utility functions in the workflow.

The commands here run a workflow using [example data](https://downgit.github.io/#/home?url=https://github.com/DeepLabCut/DeepLabCut/tree/master/examples/openfield-Pranav-2018-10-30) from the [00-DownloadData](./00-DataDownload_Optional.ipynb) notebook, but note where placeholders could be changed for a different dataset.

In [ ]:
import os; from pathlib import Path
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='workflow-deeplabcut', ("Please move to the "
                                                              + "workflow directory")
from workflow_deeplabcut.pipeline import lab, subject, session, train, model, \
                                         VideoRecording

If you previously completed the [03-Process notebook](./03-Process.ipynb), you may want to delete the contents ingested there, to avoid duplication errors.

In [ ]:
# session.Session.delete()
# train.TrainingParamSet.delete()
# train.VideoSet.delete()

## Ingestion of subjects, sessions, videos and training parameters

Refer to the `user_data` folder in the workflow.

1. Fill subject and session information in files `subjects.csv` and `sessions.csv`
2. Fill in recording and parameter information in `recordings.csv` and `config_params.csv`
    + Add both training and estimation videos to the recording list
    + Additional columns in `config_params.csv` will be treated as model training parameters
3. Run automatic scripts prepared in `workflow_deeplabcut.ingest` for ingestion: 
    + `ingest_subjects` for `subject.Subject`
    + `ingest_sessions` - for session tables `Session`, `SessionDirectory`, and `SessionNote`
    + `ingest_dlc_items` - for `VideoRecording` and `TrainingParamSet`

In [3]:
from workflow_deeplabcut.ingest import ingest_subjects, ingest_sessions, ingest_dlc_items
ingest_subjects(); ingest_sessions(); ingest_dlc_items()


---- Inserting 0 entry(s) into subject ----

---- Inserting 3 entry(s) into session ----

---- Inserting 3 entry(s) into session_directory ----

---- Inserting 3 entry(s) into session_note ----

---- Inserting 3 entry(s) into #model_training_param_set ----

---- Inserting 3 entry(s) into video_recording ----

---- Inserting 3 entry(s) into video_recording__file ----

---- Inserting 1 entry(s) into video_set ----


## Setting project variables

1. Set your root directory in your DataJoint config file, under `custom` as `dlc_root_data_dir`. For the purposes of this demo, we'll ask DeepLabCut to structure the demo config file with `load_demo_data`

In [ ]:
import datajoint as dj; dj.config.load('dj_local_conf.json')
from element_interface.utils import find_full_path
data_dir = find_full_path(dj.config['custom']['dlc_root_data_dir'], # root from config
                          'openfield-Pranav-2018-10-30')            # DLC project dir
config_path = (data_dir / 'config.yaml')
from deeplabcut.create_project.demo_data import load_demo_data
load_demo_data(config_path)

2. For this demo, we generate a copy to show pose estimation. This is `recording_id` 2 in `recordings.csv`. If you already did this in the [00-DataDownload notebook](./00-DataDownload_Optional.ipynb), skip this step.

In [5]:
vid_path = str(data_dir).replace(" ", "\ ") + '/videos/m3v1mp4'
cmd = (f'ffmpeg -n -hide_banner -loglevel error -ss 0 -t 2 -i {vid_path}.mp4 -vcodec copy '
       + f'-acodec copy {vid_path}-copy.mp4') # New video copy, first 2 seconds
os.system(cmd)

File '/Volumes/GoogleDrive/My Drive/Dev/DeepLabCut/examples/JUPYTER/openfield-Pranav-2018-10-30/videos/m3v1mp4-copy.mp4' already exists. Exiting.


256

3. Pair training video with training parameters, and launch training.

In [ ]:
key={'paramset_idx':1,'training_id':1,'video_set_id':1, 
     'project_path':'openfield-Pranav-2018-10-30/'}
train.TrainingTask.insert1(key, skip_duplicates=True)
train.ModelTraining.populate()

4. Add this model to the `Model` table and evaluate.
   - Include a user-friendly `model_name`
   - Include the relative path for the project's `config.yaml`
   - Add `shuffle` and `trainingsetindex`

In [ ]:
model.Model.insert_new_model(model_name='OpenField-5',dlc_config=config_path,
                             shuffle=1,trainingsetindex=0, paramset_idx=1,
                             model_description='Open field model trained 5 iterations')
model.ModelEvaluation.populate()

5. Add a pose estimation task, and launch pose estimation.
   - Get all primary key information for a given recording
   - Add the model and `task_mode` (i.e., load vs. trigger) to be applied
   - Add any additional analysis parameters for `deeplabcut.analyze_videos`

In [ ]:
key=(VideoRecording & 'recording_id=2').fetch1('KEY')
key.update({'model_name': 'OpenField-5', 'task_mode': 'trigger'})
analyze_params={'save_as_csv':True} # add any others from deeplabcut.analyze_videos
model.EstimationTask.insert_estimation_task(key,params=analyze_params)
model.Estimation.populate()

6. Retrieve estimated position data.

In [4]:
model.Estimation.get_trajectory(key)

scorer    OpenField-5                                                         \
bodyparts     leftear                                rightear                  
coords              x           y    z likelihood           x         y    z   
0           -0.051000  479.620728  0.0   0.273338    2.725363  6.904159  0.0   
1            0.028559  479.580170  0.0   0.270150    2.321410  7.148979  0.0   
2            0.011300  479.562500  0.0   0.267837  643.831421  4.739515  0.0   
3            0.280110  479.558044  0.0   0.273800  643.895020  4.552026  0.0   
4            0.269247  479.512573  0.0   0.267830  643.906982  4.619661  0.0   
..                ...         ...  ...        ...         ...       ...  ...   
58          -0.043869  479.548248  0.0   0.264180  644.079041  5.265405  0.0   
59           0.213278  479.605865  0.0   0.288912  643.871460  5.303852  0.0   
60          -0.011851  479.317230  0.0   0.267723  644.043091  5.202106  0.0   
61          -0.020756  479.287842  0.0   0.269066  643.995361  5.327844  0.0   
62          -0.054609  479.283264  0.0   0.266023  644.210449  5.472977  0.0   

scorer                                                                         \
bodyparts                snout                            tailbase              
coords    likelihood         x         y    z likelihood         x          y   
0           0.089934  4.700935 -7.521790  0.0   0.269598  2.385094  16.498543   
1           0.083423  5.155587 -8.236547  0.0   0.246404  1.576869  16.568169   
2           0.082102  4.967218 -6.445849  0.0   0.245935  3.414664  16.437574   
3           0.082553  5.191478 -7.312384  0.0   0.240973  2.435591  16.592468   
4           0.083153  4.552286 -7.577226  0.0   0.232977  0.996742  16.616949   
..               ...       ...       ...  ...        ...       ...        ...   
58          0.084058  6.878569 -8.191770  0.0   0.247327  0.691330  16.979179   
59          0.086554  7.673346 -9.163953  0.0   0.207005  2.231732  17.397644   
60          0.083778  6.811278 -9.130725  0.0   0.232106  2.037935  17.634174   
61          0.085517  6.878219 -9.760260  0.0   0.232977  0.569980  17.509853   
62          0.085272  7.383186 -9.041880  0.0   0.223843  2.449749  17.441238   

scorer                     
bodyparts                  
coords       z likelihood  
0          0.0   0.227193  
1          0.0   0.219187  
2          0.0   0.199375  
3          0.0   0.219349  
4          0.0   0.223073  
..         ...        ...  
58         0.0   0.240952  
59         0.0   0.211547  
60         0.0   0.229108  
61         0.0   0.241432  
62         0.0   0.214529  

[63 rows x 16 columns]

## Summary and next step

+ This notebook runs through the workflow in an automatic manner.

+ The next notebook [06-Drop](06-Drop_Optional.ipynb) shows how to drop schemas and tables if needed.